In [1]:
import ROOT
import math
%jsroot on

Welcome to JupyROOT 6.16/00


### Neutron distributions

In [2]:
eventless_file = ROOT.TFile('data/output.root')

In [3]:
# Preparing histograms
xy_hist = ROOT.TH2D('xy_hist', 'Neutrons angle distribution', 60, -0.2, 0.1, 60, -0.15, 0.15)
x_hist = ROOT.TH1D('x_hist', 'Neutrons horizontal angle distribution', 60, -0.2, 0.1)
y_hist = ROOT.TH1D('y_hist', 'Neutrons vertical angle distribution', 60, -0.15, 0.15)
e_hist = ROOT.TH1D('e_hist', 'Neutrons energy distribution; [GeV]', 140, 0, 70)

neutrons_only = ROOT.TCut('pdg==2112')

In [4]:
# Reading a tree from the file
tree = eventless_file.Get('eventless').Get('tree')

# Filling the histograms from the tree
canvas = ROOT.TCanvas('canvas', 'canvas',600,400)
tree.Draw('asin(py/p):atan(px/pz)>>xy_hist', neutrons_only)
tree.Draw('atan(px/pz)>>x_hist', neutrons_only)
tree.Draw('asin(py/p)>>y_hist', neutrons_only)
tree.Draw('tot_e>>e_hist', neutrons_only)

14150

In [5]:
e_hist.Draw()
canvas.Draw()

In [6]:
canvas0 = ROOT.TCanvas('canvas0', 'canvas0',760,700)
canvas0.Divide(2,2)

In [7]:
is_log_scale = True

l_margin = 0.12
r_margin = 0.12

pad = canvas0.cd(1)
pad.SetLogx(False)
pad.SetLogy(is_log_scale)
pad.SetLogz(False)
pad.SetLeftMargin(l_margin)
pad.SetRightMargin(r_margin)
x_hist.SetXTitle('Horizontal angle (x) [rad]')
x_hist.SetStats(False)
x_hist.SetFillColor(ROOT.kBlue+1)
x_hist.SetBarWidth(1.)
x_hist.SetBarOffset(0.)
x_hist.Draw('bar')

pad = canvas0.cd(2)
pad.SetLogx(False)
pad.SetLogy(False)
pad.SetLogz(is_log_scale)
pad.SetLeftMargin(l_margin)
pad.SetRightMargin(r_margin)
xy_hist.SetXTitle('Horizontal angle (x) [rad]')
xy_hist.SetYTitle('Vertical angle (y) [rad]')
xy_hist.SetStats(False)
xy_hist.Draw('lego2z')

pad = canvas0.cd(3)
pad.SetLogx(False)
pad.SetLogy(False)
pad.SetLogz(is_log_scale)
pad.SetLeftMargin(l_margin)
pad.SetRightMargin(r_margin)
xy_hist.SetStats(False)
xy_hist.Draw('colz')

pad = canvas0.cd(4)
pad.SetLogx(is_log_scale)
pad.SetLogy(False)
pad.SetLogz(False)
pad.SetLeftMargin(l_margin)
pad.SetRightMargin(r_margin)
y_hist.GetXaxis().SetTitleOffset(1.5)
y_hist.SetXTitle('Vertical angle (y) [rad]')
y_hist.SetStats(False)
y_hist.SetFillColor(ROOT.kBlue+1)
y_hist.SetBarWidth(1.)
y_hist.SetBarOffset(0.)
y_hist.Draw('hbar')

canvas0.Draw()

Horizontal angle (x):
$
\begin{align}
\alpha_h = arctan(\dfrac{p_x}{p_z})
\end{align}
$

Vertical angle (y):
$
\begin{align}
\alpha_v = arcsin(\dfrac{p_y}{p})
\end{align}
$

###   Open charm

In [8]:
file_sm0 = ROOT.TFile('data/open_charm.root')
file_sm1 = ROOT.TFile('data/open_charm_sm.root')

In [9]:
dirs = [file_sm0.Get('opencharm'), file_sm1.Get('opencharm')]
getVarName = lambda name: name.replace(' ', '_').replace('/', '_').lower()
for name in [key.GetName() for key in dirs[0].GetListOfKeys()]:
    varName = getVarName(name)
    exec("{varName} = [d.Get('{name}') for d in dirs]".format(varName=varName, name=name))

In [10]:
canvas1 = ROOT.TCanvas('canvas1', 'canvas1', 350, 350)

canvas2 = ROOT.TCanvas('canvas2', 'canvas2', 700, 350)
canvas2.Divide(2)

def draw_hist(hist, log=False):
    canvas1.cd().SetLogy(log)
    hist.SetStats(False)
    hist.Draw()
    canvas1.Draw()
    
def draw_hists(hists, log=False, opt=''):
    if 'true MC' not in hists[0].GetTitle():
        hists[0].SetTitle(hists[0].GetTitle() + ' (true MC)')
    if 'smear' not in hists[1].GetTitle():
        hists[1].SetTitle(hists[1].GetTitle() + ' (smeared)')
    for i in range(2):
        if opt=='same':
            hists[i].SetLineColor(ROOT.kRed)
        canvas2.cd(i+1).SetLogy(log)
        hists[i].SetStats(False)
        hists[i].Draw(opt)
    canvas2.Draw()

In [11]:
d0_pt[0].SetXTitle('[GeV]')
draw_hist(d0_pt[0])

In [12]:
def expo(x, par):
    return math.exp(par[0] + x[0]*par[1])


def gaus(x, par):
    if par[2]==0:
        return 0
    return par[0]*math.exp(-0.5*math.pow(((x[0]-par[1])/par[2]),2))


def gausPlusExpo(x, par):
    if par[4]==0:
        return 0
    return math.exp(par[0] + x[0]*par[1]) + par[2]*math.exp(-0.5*math.pow(((x[0]-par[3])/par[4]),2))


fit_func = ROOT.TF1('fit_func', gausPlusExpo, 1.7, 2.4, 5)
tf_expo = ROOT.TF1('tf_expo', expo, 1.7, 2.4, 2)
tf_gaus = ROOT.TF1('tf_gaus', gaus, 1.7, 2.4, 3)


fit_func.SetParameter(0, 12)
fit_func.SetParameter(1, -1.5)

fit_func.SetParameter(2, 2e4)
fit_func.SetParameter(3, 1.87)
fit_func.SetParameter(4, 0.01)

fit_func.SetParLimits(2, 1e4, 5e4)
fit_func.SetParLimits(3, 1.85, 1.89)

if 'true MC' not in d0_mass[0].GetTitle():
    d0_mass[0].SetTitle(d0_mass[0].GetTitle() + ' (true MC)')
if 'smear' not in d0_mass[1].GetTitle():
    d0_mass[1].SetTitle(d0_mass[1].GetTitle() + ' (smeared)')

canvas2.cd(1).SetLogy(False)
d0_mass[0].SetStats(False)
d0_mass[0].SetXTitle('[GeV]')
d0_mass[0].Draw()

canvas2.cd(2).SetLogy(False)
d0_mass[1].Fit(fit_func, 'BQR')
d0_mass[1].SetMarkerStyle(8)
d0_mass[1].SetMarkerSize(0.7)
d0_mass[1].SetStats(False)
d0_mass[1].SetXTitle('[GeV]')
d0_mass[1].Draw('p')

fit_parameters = fit_func.GetParameters()
tf_expo.SetParameters(fit_parameters)

tf_expo.SetLineColor(ROOT.kAzure+7)
tf_expo.Draw('same')

legend = ROOT.TLegend(0.5,0.65,0.87,0.87);
legend.AddEntry(d0_mass[1],"Data","p");
legend.AddEntry(tf_expo,"Fit: background","l");
legend.AddEntry(tf_gaus,"Fit: D^{0}","l");
legend.Draw()

canvas2.Draw()

In [13]:
dst_pt[0].SetXTitle('[GeV]')
draw_hist(dst_pt[0])

In [14]:
dst_dm[0].SetXTitle('[GeV]')
dst_dm[1].SetXTitle('[GeV]')
draw_hists(dst_dm)

#### Cut:
$Mass(D^{0})\in\left[1.79;1.93\right] $

In [15]:
dst_dm_mcut[0].SetXTitle('[GeV]')
dst_dm_mcut[1].SetXTitle('[GeV]')
draw_hists(dst_dm_mcut)